## La Liga: A Brief Analysis ##

# 1단계: 라이브러리 임포트 (Import Libraries)

In [5]:
# 1-1. 라이브러리 임포트 (Import Libraries)
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

# 1-2. 데이터 로드 및 'date' 컬럼 타입 변환 (Load Data and Convert 'date' column)
df = pd.read_csv('la_liga_2014_2025_all_matches_final.csv')
df['date'] = pd.to_datetime(df['date'])

# 1-3. 정확한 시즌 라벨링 함수 정의 (Define Accurate Season Labeling Function)
# 제공해주신 시즌 정보를 바탕으로, 날짜가 어느 시즌에 속하는지 구분합니다.
season_end_dates = {
    '2014-2015': '2015-05-24', '2015-2016': '2016-05-16', '2016-2017': '2017-05-22',
    '2017-2018': '2018-05-21', '2018-2019': '2019-05-27', '2019-2020': '2020-07-20',
    '2020-2021': '2021-05-24', '2021-2022': '2022-05-23', '2022-2023': '2023-06-05',
    '2023-2024': '2024-05-27', '2024-2025': '2025-05-26'
}
# 날짜 문자열을 datetime 객체로 변환합니다.
season_end_dates_dt = {season: pd.to_datetime(end_date) for season, end_date in season_end_dates.items()}

def get_season(date):
    for season, end_date in season_end_dates_dt.items():
        if date <= end_date:
            # 해당 시즌의 시작일을 찾습니다. (이전 시즌의 종료일 + 1일)
            previous_season_year = int(season.split('-')[0]) - 1
            previous_season = f"{previous_season_year}-{previous_season_year+1}"
            start_date = season_end_dates_dt.get(previous_season, pd.to_datetime('1900-01-01')) + pd.Timedelta(days=1)
            
            if date >= start_date:
                return season
    return "Unknown"

df['season'] = df['date'].apply(get_season)

print("Data loading and season labeling complete.")

# 1-4. 시즌별 경기 수 확인 (Verify Match Count per Season)
# La Liga는 20개 팀이 홈&어웨이로 경기를 치르므로, 한 시즌은 (20 * 19) / 2 = 190경기가 아닌, 팀당 38경기씩 총 380경기가 맞습니다.
# (팀당 38경기 * 20팀) / 2 = 380 경기.
# 이 검증을 통해 데이터가 누락 없이 잘 포함되었는지 확인합니다.
match_counts = df.groupby('season').size().reset_index(name='match_count')
print("\n--- Season by Match Count ---")
print(match_counts)

# 시각화를 통해 한눈에 확인
fig_match_counts = px.bar(match_counts, x='season', y='match_count', text_auto=True,
                          title='Number of Matches per Season')
fig_match_counts.add_hline(y=380, line_dash="dash", line_color="red", annotation_text="Standard Line (380 Matches)")
fig_match_counts.show()

Data loading and season labeling complete.

--- Season by Match Count ---
       season  match_count
0   2014-2015          380
1   2015-2016          380
2   2016-2017          380
3   2017-2018          380
4   2018-2019          380
5   2019-2020          380
6   2020-2021          380
7   2021-2022          380
8   2022-2023          380
9   2023-2024          380
10  2024-2025          380


### 1. EDA 결과 해석 (Interpretation of EDA Results)

- English: The dataset is clean and robust, containing 4,180 match records with 20 columns and no missing values. 

- Korean: 이 데이터셋은 4,180개의 경기 기록을 담고 있으며 결측치가 전혀 없는 매우 깨끗한 데이터입니다.

# 2단계: 탐색적 데이터 분석 (EDA - Exploratory Data Analysis)

In [6]:
# 2-1. 데이터 기본 정보 확인 (Initial Data Inspection)
print("--- Data Basic Information ---")
df.info()

print("\n\n--- Descriptive Statistics ---")
print(df.describe())

print("\n\n--- Missing Value Check ---")
print(df.isnull().sum())

# 2-2. 주요 변수 간 상관관계 분석 (Correlation Analysis of Key Variables)
corr_matrix = df[['home_goals', 'away_goals', 'home_xg', 'away_xg', 'home_shots', 'away_shots', 'home_sot', 'away_sot', 'home_deep', 'away_deep', 'home_ppda', 'away_ppda']].corr()

fig_corr = px.imshow(corr_matrix, text_auto=True, aspect="auto",
                     title="Correlation Heatmap of Key Variables")
fig_corr.show()

--- Data Basic Information ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4180 entries, 0 to 4179
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   match_id       4180 non-null   int64         
 1   date           4180 non-null   datetime64[ns]
 2   home_team      4180 non-null   object        
 3   away_team      4180 non-null   object        
 4   home_goals     4180 non-null   int64         
 5   away_goals     4180 non-null   int64         
 6   home_xg        4180 non-null   float64       
 7   away_xg        4180 non-null   float64       
 8   home_shots     4180 non-null   int64         
 9   away_shots     4180 non-null   int64         
 10  home_sot       4180 non-null   int64         
 11  away_sot       4180 non-null   int64         
 12  home_deep      4180 non-null   int64         
 13  away_deep      4180 non-null   int64         
 14  home_ppda      4180 non-null   float64   

### 2. Correlation Heatmap of Key Variables (주요 변수 간 상관관계 히트맵)

#### English
	•	The strongest relationships appear between expected goals (xG) and shots on target (SoT) for both home and away teams (≈0.66–0.67).
	•	This is intuitive: the more accurate shots a team produces, the higher its expected goals.
	•	Also, total shots and shots on target show a strong correlation (≈0.64–0.65), since SoT is a subset of shots.
	•	Finally, away xG and away goals correlate at 0.64, showing that xG is a meaningful predictor of actual scoring.

#### Korean (한국어)
	•	가장 강한 상관관계는 기대 득점(xG) 과 유효 슈팅(SoT) 사이에서 나타났습니다 (약 0.66–0.67).
	•	이는 직관적으로 타당합니다. 유효 슈팅이 많을수록 득점 기대값이 높아지기 때문입니다.
	•	또한 총 슈팅 수와 유효 슈팅 수 역시 강한 상관관계(0.64~0.65)를 보였습니다. 이는 유효 슈팅이 슈팅의 부분집합이기 때문입니다.
	•	마지막으로, 원정 xG와 원정 득점 간 상관관계가 0.64로 나타나, xG가 실제 득점을 설명하는 데 중요한 지표임을 확인할 수 있습니다. 

#  3단계: La Liga 전체 트렌드 분석 (League-wide Trend Analysis)

In [7]:
# 3-1. 시즌별 경기당 평균 득점 추이 (Seasonal Goals per Match Trend)
season_goals = df.groupby("season").apply(
    lambda x: (x["home_goals"] + x["away_goals"]).sum() / len(x)
).reset_index(name="goals_per_match")

fig_season_goals = px.line(
    season_goals,
    x="season",
    y="goals_per_match",
    markers=True,
    title="Seasonal Average Goals per Match",
    labels={"season": "Season", "goals_per_match": "Goals per Match"}
)
fig_season_goals.show()

# 3-2. 시즌별 홈/원정 승률 변화 (Seasonal Home/Away Win Rate Changes)
df["result"] = np.where(
    df["home_goals"] > df["away_goals"], "Home Win",
    np.where(df["home_goals"] < df["away_goals"], "Away Win", "Draw")
)
res_counts = df.groupby(["season", "result"]).size().unstack(fill_value=0)
res_rates = res_counts.div(res_counts.sum(axis=1), axis=0)

fig_win_rate = go.Figure()
fig_win_rate.add_bar(name='Home Win', x=res_rates.index, y=res_rates['Home Win'])
fig_win_rate.add_bar(name='Draw', x=res_rates.index, y=res_rates['Draw'])
fig_win_rate.add_bar(name='Away Win', x=res_rates.index, y=res_rates['Away Win'])

fig_win_rate.update_layout(
    barmode='stack',
    title="Changes in Home/Draw/Away Win Rates by Season",
    xaxis_title="Season",
    yaxis_title="Rate",
    yaxis_tickformat='.0%'
)
fig_win_rate.show()

/var/folders/yl/3cpbqsq96s70dm9lcf8p9c_00000gn/T/ipykernel_1422/3018619177.py:2: FutureWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



3. Seasonal Average Goals per Match (시즌별 평균 득점 추세)

- **English**: This line chart shows the average number of goals scored per match across La Liga seasons from 2014–15 to 2024–25.  
  The peak was in the 2016–17 season (around 2.93 goals per match), followed by a decline until 2019–20. Recently, goal averages have stabilized around 2.6.  
  This indicates cyclical patterns in offensive intensity across the league.  

- **한국어**: 이 선 그래프는 2014–15 시즌부터 2024–25 시즌까지 라리가 경기당 평균 득점 수를 보여줍니다.  
  2016–17 시즌에 약 2.9득점으로 정점을 찍은 뒤 2019–20 시즌까지 감소했으며, 최근에는 약 2.6득점 수준에서 안정세를 보입니다.  
  이는 라리가 전체 공격 강도가 일정한 주기적 변화를 겪고 있음을 의미합니다.  


---
 3. Home/Draw/Away Win Rates (홈/무/원정 승률 변화)
- **English**  
The stacked bar chart illustrates the seasonal distribution of home wins, draws, and away wins.  
Normally, home teams achieve the highest share of wins, reflecting the well-known *home advantage*.  

- However, in the **2020–2021 season**, the home win rate declined significantly, while away wins increased.  
This coincided with the COVID-19 pandemic, during which matches were played behind closed doors.  
The absence of fans diminished the psychological and environmental benefits usually enjoyed by home teams.  

---

- **Korean (한국어)**  
이 누적 막대 그래프는 시즌별 **홈팀 승리 / 무승부 / 원정팀 승리 비율**을 보여줍니다.  
일반적으로 홈팀의 승률이 가장 높으며, 이는 전통적인 *홈 어드밴티지*를 반영합니다.  

- 그러나 **2020–2021 시즌**에는 홈팀 승률이 크게 하락하고 원정팀 승률이 상승했습니다.  
이 시기는 코로나19로 인해 **무관중 경기**가 치러진 시즌으로, 관중 응원과 같은 홈 어드밴티지가 줄어든 결과로 해석할 수 있습니다.  

---

📌 **Portfolio Note**  
- The drop in home win rates in **2020–2021** aligns with the absence of fans during the COVID-19 pandemic.  
- Interpreting this change as a shift in *home advantage* makes the analysis more professional and research-oriented.  

# 4단계: Big 3 공격효율성 분석 (Big 3 Attack Efficiency Trend Analysis (GF – xGF per Match)
)

In [9]:
# 4-1. 데이터 재구성 (Restructure Data)
home_df = df[['season', 'home_team', 'home_goals', 'home_xg']].rename(columns={'home_team': 'team', 'home_goals': 'GF', 'home_xg': 'xGF'})
away_df = df[['season', 'away_team', 'away_goals', 'away_xg']].rename(columns={'away_team': 'team', 'away_goals': 'GF', 'away_xg': 'xGF'})
team_df = pd.concat([home_df, away_df], ignore_index=True)

# 4-2. 이름 표준화 함수 및 딕셔너리 정의 (Define Name Standardization)
big3_alias = {
    "Real Madrid": ["Real Madrid"],
    "Barcelona":   ["Barcelona", "FC Barcelona"],
    "Atletico Madrid": ["Atletico Madrid", "Atlético Madrid"]
}
def standardize_name(team_name):
    for standard_name, aliases in big3_alias.items():
        if team_name in aliases:
            return standard_name
    return team_name
team_df['team_std'] = team_df['team'].apply(standardize_name)

# 4-3. Big 3 팀 필터링 및 시즌별 집계 (Filter and Aggregate)
big3_teams = ["Real Madrid", "Barcelona", "Atletico Madrid"]
big3_df = team_df[team_df['team_std'].isin(big3_teams)]
season_team_agg = big3_df.groupby(['season', 'team_std']).agg(
    GF_total=('GF', 'sum'),
    xGF_total=('xGF', 'sum'),
    matches=('team_std', 'size')
).reset_index().rename(columns={'team_std': 'team'})

# 4-4. 경기당 지표 및 공격 효율성 계산 (Calculate Metrics)
season_team_agg['GF_per_match'] = season_team_agg['GF_total'] / season_team_agg['matches']
season_team_agg['xGF_per_match'] = season_team_agg['xGF_total'] / season_team_agg['matches']
season_team_agg['attack_efficiency'] = season_team_agg['GF_per_match'] - season_team_agg['xGF_per_match']

# 4-5. [수정됨] 공격 효율성 시각화 (REVISED: Visualize Attack Efficiency with Labels)
fig_efficiency = px.bar(
    season_team_agg,
    x="season",
    y="attack_efficiency",
    color="team",
    barmode='group',
    title="Big 3 Attack Efficiency Trend (GF - xGF per Match)",
    labels={
        "season": "Season",
        "attack_efficiency": "Attack Efficiency (GF - xGF per Match)",
        "team": "Team"
    },
    text_auto=True  # 막대에 텍스트를 자동으로 추가하는 기능
)

# 텍스트 라벨의 소수점 자릿수와 위치를 조정하여 가독성을 높입니다.
fig_efficiency.update_traces(texttemplate='%{y:.2f}', textposition='outside')
fig_efficiency.add_hline(y=0, line_width=2, line_dash="dash", line_color="black")
fig_efficiency.show()

### 4. Graph Interpretation (그래프 해석)

# **Definition**  
  - Attack Efficiency = GF_per_match − xGF_per_match  

# 🔵 Atletico Madrid
	-	Mostly positive values until late 2010s → slightly more goals than expected.
	-	2019-20 & 2024-25: Negative values → attack efficiency dropped.
	-	Overall stable, but recent decline in finishing quality observed.

# 🔴 Barcelona
	-	2014–2017: Large positive margin → Messi’s prime years, highly efficient finishing.
	-	2018 onward: Gradual decline, with 2022-23 below zero → inefficient finishing.
	-	Recently slight recovery, but no longer dominant as in the past.

# 🟢 Real Madrid
	-	2014–2017: Strong positive values → Ronaldo era, elite finishing efficiency.
	-	2018 onward: Gradual decline, turning negative in 2024-25.
	-	Reflects post-Ronaldo transition and reliance on new attacking structures.
===

# 🔵 Atletico Madrid
	•	대부분 시즌에서 +값(0 이상) → 기대득점 대비 실제 득점이 조금 더 많았음.
	•	하지만 2019-20, 2024-25 시즌에는 마이너스로 떨어짐 → 공격 효율이 기대치보다 낮았음을 의미.
	•	전반적으로 안정적인 득점 효율을 보여줬으나, 최근 시즌들에는 득점력 하락 조짐이 있음.

# 🔴 Barcelona
	•	2014-2017 시즌에는 GF-xGF가 크게 양수 → 메시 전성기 시절, 기대득점보다 훨씬 많은 득점을 기록.
	•	그러나 2018-19 이후 점차 줄어들어 2022-23 시즌에는 음수 → 기대득점보다 덜 득점하는 비효율적인 공격력.
	•	최근 다시 소폭 회복했으나, 과거만큼의 압도적 효율은 사라짐.

# 🟢 Real Madrid
	•	2014-2017 시즌 압도적 양수 → 호날두 시절, 기회보다 훨씬 많은 득점 (골 결정력 최강).
	•	2018-2019 이후 점차 하락, 최근 시즌(2024-25)에서는 오히려 음수 → 기대득점보다 덜 넣는 모습.
	•	이는 호날두 이탈 + 벤제마 이후 공격 전환 과정에서 나타난 변화로 해석 가능.